Neste notebook importarei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA desde a temporada 1984-1985(Quando Michael Jordan entrou como novato) até o fim da temporada 2020-2021
para analisar como seria a tabela de classificação da temporadas regulares de 2010-2011(Lebron entra no miami heat) até o fim da temporada 2017-2018(Lebron entra no Lakers),caso Michael Jordan entrasse na NBA na temporada 2010-2011

(Lembrar de mudar isso )

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import pandas as pd

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'}

times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK',
        'Charlotte Bobcats' : 'CHA',
        'Vancouver Grizzlies' : 'VAN',
        'New Orleans Hornets' : 'NOH',
        'New Orleans/Oklahoma City Hornets' : 'NOK'}

anos = list(range(1980,2021))

todos_jogadores = []
todos_mvps = []

## Funções para pegar os dados de mvps e os dados dos times

In [2]:
def pegar_mvps(url, dados_mvps):
    
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    mvps_html = soup.find('tbody').findAll('tr')
    cont_rank = 1
    for mvp_html in mvps_html:

        dados_mvp = {}
        
        dados_mvp['rank'] = cont_rank
        cont_rank += 1
        
        nome_jogador = mvp_html.find('td', {'data-stat':'player'}).find('a').getText()
        dados_mvp['team'] = mvp_html.find('td', {'data-stat':'team_id'}).getText()
        dados_mvp['award_share'] = float(mvp_html.find('td', {'data-stat':'award_share'}).getText())
        
        dados_mvps[nome_jogador] = dados_mvp


def pegar_times_conferencia(soup, dados_times, sufixo):
    times_html = soup.find('table', id = f'confs_standings_{sufixo}')
    
    if times_html == None:
        times_html = soup.find('table', id = f'divs_standings_{sufixo}').find('tbody').findAll('tr')
    else:
        times_html = soup.find('table', id = f'confs_standings_{sufixo}').find('tbody').findAll('tr')
    
    
    cont = 1
    for time_html in times_html:
        if(time_html.get('class')[0] != 'thead'):
            dados_time = {}
            nome_time = times[time_html.find('th').find('a').getText()]


            dados_time['conf_rank'] = cont
            cont+=1

            time_stats_html = time_html.findAll('td')

            for stat in time_stats_html:
                try:
                    dados_time[stat.get('data-stat')] = float(stat.getText())
                except ValueError:
                    dados_time[stat.get('data-stat')] = 0

            dados_times[nome_time] = dados_time
        
def pegar_times(url,dados_times):
    try:
        req = Request(url_dados_time, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    print(ano)
    "Pegando os dados da Conferência Leste"
    pegar_times_conferencia(soup,dados_times, 'E')
    "Pegando os dados da Conferência Oeste"
    pegar_times_conferencia(soup,dados_times, 'W')
    

## Funções para carregar dados dos jogadores

In [3]:
def primeiro_scraping_jogadores(url, dados_jogadores, ano):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
   
    for jogador_html in jogadores_html:
        dados_jogador = {}
        if jogador_html.get('class')[0] != 'thead':
            
            if jogador_html.find('td', {'data-stat':'team_id'}).a == None:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).getText()
            else:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).a.getText()
            
            parametro_player = ''.join(jogador_html.find('td', {'data-stat':'player'}).find('a').getText().split(' '))
            dados_jogador['player_season_team'] = parametro_player +'_' + str(ano) + '_' + sigla_time
            dados_jogador['player'] = jogador_html.find('td', {'data-stat':'player'}).find('a').getText()
            dados_jogador['season'] = ano
            dados_jogador['pos'] = jogador_html.find('td', {'data-stat':'pos'}).getText()
            dados_jogador['age'] = int( jogador_html.find('td', {'data-stat':'age'}).getText() )
            dados_jogador['team'] = sigla_time
            
            for stat in jogador_html.findAll('td')[4:]:
                
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())
                    
                except ValueError:
                    if(stat.getText() != ''):
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    else:
                        dados_jogador[stat.get('data-stat')] = 0.0
            dados_jogador['mp_pergame'] = round(dados_jogador['mp']/dados_jogador['g'], 1)
            dados_jogadores.append(dados_jogador)

        
def intermedirario_scraping_jogadores(url, dados_jogadores):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
    
    i = 0
    
    for jogador_html in jogadores_html:
        dados_jogador = dados_jogadores[i]
        
        if jogador_html.get('class')[0] != 'thead':
            i += 1
            
            stats_jogador = jogador_html.findAll('td')[8:]
            for stat in stats_jogador:
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())

                except ValueError:
                    if(stat.getText() == ''):
                        dados_jogador[stat.get('data-stat')] = 0.0
                    elif stat.getText()[0] == '-':
                        dados_jogador[stat.get('data-stat')] =(-1)*float('0' + stat.getText()[1:])
                    else:
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    
            

In [4]:
for ano in anos:
    "Nesta seção do código serão carregadas as páginas html"
    
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    url_dados_totais_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_totals.html'
    url_dados_por_jogo_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_game.html'
    url_dados_por_36min_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_minute.html'
    url_dados_por_100pos_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_poss.html'
    url_dados_avancados_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_advanced.html'
    url_votacao_mvp = f'https://www.basketball-reference.com/awards/awards_{ano}.html'
    
    
    "Aqui será carregado os dados dos candidatos a MVP relativos a este ano"
    
    dados_mvps = {}
    pegar_mvps(url_votacao_mvp,dados_mvps)
    
   
    
    "Aqui serão carregados os dados do time"
    
    "Aqui é a variável que armazenará os dados do time relativos àquele ano"
    dados_times = {}
    
    pegar_times(url_dados_time, dados_times)
    
    
    "Finalmente, serão carregados os dados dos jogadores"
    
    dados_jogadores = []
    
    primeiro_scraping_jogadores(url_dados_totais_jogador, dados_jogadores, int(ano))
    
    intermedirario_scraping_jogadores(url_dados_por_jogo_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_36min_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_100pos_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_avancados_jogador, dados_jogadores)
    
    nomes_dos_mvps = list(dados_mvps.keys())
    #Como o recorte é da década de 80 para cá e o Lakers não mudou de nomes desde então, estou usando essa pequena gambiarra
    colunas_time = list(dados_times['LAL'].keys())
    
    for dados_jogador in dados_jogadores:
        sigla_time = dados_jogador['team']
        
        if sigla_time != 'TOT': 
            
            if sigla_time == 'CHH':
                sigla_time = 'CHO'
            
            dict_time = dados_times[sigla_time]

            for coluna_time in colunas_time:
                
                dados_jogador['team_' + coluna_time] = dict_time[coluna_time]

            if dados_jogador['player'] in nomes_dos_mvps:
                dados_jogador['mvp_candidate'] = True
                dict_mvp = dados_mvps[dados_jogador['player']]
                rank_mvp = dict_mvp['rank']
                
                
                if(rank_mvp<=4):
                    dados_jogador['mvp_podium'] = True
                    if(rank_mvp==1):
                        dados_jogador['mvp_winner'] = True
                    else:
                        dados_jogador['mvp_winner'] = False
                else:
                    dados_jogador['mvp_podium'] = False
                    dados_jogador['mvp_winner'] = False
                
                dados_jogador['mvp_votes_share'] = dict_mvp['award_share']
                
                parametro_player = ''.join(dados_jogador['player'].split(' '))

                parametro_plt = parametro_player + '_' + str(ano) + '_' + dados_jogador['team']
                todos_mvps.append({'player_season_team': parametro_plt,'season':ano ,'rank': rank_mvp,
                                   'mvp_votes_share': dict_mvp['award_share'], 'mvp_candidate':dados_jogador['mvp_candidate'],
                                   'mvp_podium':dados_jogador['mvp_podium'], 'mvp_winner':dados_jogador['mvp_winner']})
            else:
                dados_jogador['mvp_candidate'] = False
                dados_jogador['mvp_podium'] = False
                dados_jogador['mvp_winner'] = False
                dados_jogador['mvp_votes_share'] = 0.0
            

    todos_jogadores += dados_jogadores

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [5]:
dataset1 = pd.DataFrame(todos_jogadores)
dataset1

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,KareemAbdul-Jabbar_1980_LAL,Kareem Abdul-Jabbar,1980,C,32,LAL,82,0.0,3143,835,...,22.0,0.732,0.0,115.1,109.2,5.40,True,True,True,0.665
1,TomAbernethy_1980_GSW,Tom Abernethy,1980,PF,25,GSW,67,0.0,1222,153,...,58.0,0.293,36.0,103.6,108.0,-4.20,False,False,False,0.000
2,AlvanAdams_1980_PHO,Alvan Adams,1980,C,25,PHO,75,0.0,2168,465,...,27.0,0.671,5.0,111.1,107.5,3.25,False,False,False,0.000
3,TinyArchibald_1980_BOS,Tiny Archibald,1980,PG,31,BOS,80,80.0,2864,383,...,21.0,0.744,0.0,113.5,105.7,7.37,True,False,False,0.009
4,DennisAwtrey_1980_CHI,Dennis Awtrey,1980,C,31,CHI,26,0.0,560,27,...,52.0,0.366,19.0,107.5,110.2,-2.63,False,False,False,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20945,TraeYoung_2020_ATL,Trae Young,2020,PG,21,ATL,60,60.0,2120,546,...,47.0,0.299,33.0,111.8,119.7,-7.71,False,False,False,0.000
20946,CodyZeller_2020_CHO,Cody Zeller,2020,C,27,CHO,58,39.0,1341,251,...,42.0,0.354,29.0,102.9,109.6,-7.03,False,False,False,0.000
20947,TylerZeller_2020_SAS,Tyler Zeller,2020,C,30,SAS,2,0.0,4,1,...,39.0,0.451,20.0,114.1,115.2,-0.65,False,False,False,0.000
20948,AnteŽižić_2020_CLE,Ante Žižić,2020,C,23,CLE,22,0.0,221,41,...,46.0,0.292,33.0,106.9,114.8,-7.77,False,False,False,0.000


In [6]:
dataset2 = pd.DataFrame(todos_mvps)
dataset2

,player_season_team,season,rank,mvp_votes_share,mvp_candidate,mvp_podium,mvp_winner
0,KareemAbdul-Jabbar_1980_LAL,1980,1,0.665,True,True,True
1,TinyArchibald_1980_BOS,1980,5,0.009,True,False,False
2,LarryBird_1980_BOS,1980,4,0.068,True,True,False
3,JuliusErving_1980_PHI,1980,2,0.143,True,True,False
4,GeorgeGervin_1980_SAS,1980,3,0.086,True,True,False
...,...,...,...,...,...,...,...
671,KawhiLeonard_2020_LAC,2020,5,0.166,True,False,False
672,DamianLillard_2020_POR,2020,8,0.023,True,False,False
673,ChrisPaul_2020_OKC,2020,7,0.026,True,False,False
674,PascalSiakam_2020_TOR,2020,10,0.017,True,False,False


In [8]:
dataset1.to_csv('./data/dataset_jogadores.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset2.to_csv('./data/dataset_mvps.csv', sep=';', index = False, encoding = 'utf-8-sig')